In [1]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

In [2]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [3]:
# Fetch the sdRDM schema
lib = DataModel.from_markdown("./model.md")
lib.Root.meta_tree()

Root
├── id
├── value
├── nested
│   └── 0
│       ├── id
│       ├── another_value
│       └── value
└── some_values


ClassNode(/Root, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [4]:
# Create tables for the sdRDM schema
create_tables(
    db_connector=db,
    model=lib.Root,
    markdown_path="./model.md",
)


🚀 Creating tables for data model Root
│
├── Table __model_meta__ not existing. Adding to DB!
├── Added table model 'Root' to __model_meta__ table
├── Model 'Root' already registered. Skipping.
├── Created table 'Root'
├── Created table 'Root_some_values'
├── Added table model 'Root_nested' to __model_meta__ table
├── Created table 'Root_nested'
├── Added primary key 'Root_id' to table Root
├── Added primary key 'Root_nested_id' to table Root_nested
├── Added foreign key 'Root_id'(Root) to table Root_some_values
├── Added foreign key 'Root_id'(Root) to table Root_nested
│
╰── 🎉 Created all tables for data model Root



In [5]:
# We are now seeking a way to insert an instance of 
# the sdRDM schema into the database.
print(db.connection.table("__model_meta__"))
print(db.connection.table("Root"))
print(db.connection.table("Root_nested"))
print(db.connection.table("Root_some_values"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓
┃ table       ┃ specifications                             ┃ github_url ┃ commit_hash ┃ part_of ┃ obj_name ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩
│ !string     │ json                                       │ string     │ string      │ string  │ string   │
├─────────────┼────────────────────────────────────────────┼────────────┼─────────────┼─────────┼──────────┤
│ Root        │ {'objects': [...], 'enums': [...], ... +3} │ NULL       │ NULL        │ NULL    │ Root     │
│ Root_nested │ NULL                                       │ NULL       │ NULL        │ Root    │ Nested   │
└─────────────┴────────────────────────────────────────────┴────────────┴─────────────┴─────────┴──────────┘



┏━━━━━━━━━┳━━━━━━━━━┓
┃ value   ┃ Root_id ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│ float64 │ !string │
└─────────┴─────────┘



┏━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ another_value ┃ value   ┃ Root_nested_id ┃ Root_id ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string        │ float64 │ !string        │ string  │
└───────────────┴─────────┴────────────────┴─────────┘



┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ some_values ┃ Root_id ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ !string     │ string  │
└─────────────┴─────────┘



In [6]:
# Create datasets using the API that has been directly generated from the DB
model = db.get_table_api("Root")

dataset1 = model(value=100.0, some_values=["Hello", "World"])
dataset1.add_to_nested(another_value="something")
dataset1.add_to_nested(another_value="something else")

print(dataset1)

Root
├── id = root0
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested0
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = nested1
│           └── another_value = something else
└── some_values = [Hello, World, ...]



In [7]:
# And another one
dataset2 = model(value=20.0, some_values=["Whats", "up"])
dataset2.add_to_nested(another_value="hi there")
dataset2.add_to_nested(another_value="this is dataset 2")

print(dataset2)

Root
├── id = root1
├── value = 20.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested2
│   │       └── another_value = hi there
│   └── 1
│       └── Nested
│           ├── id = nested3
│           └── another_value = this is dataset 2
└── some_values = [Whats, up, ...]



In [8]:
# Insert the new dataset into the database
db.insert(dataset1, dataset2, verbose=True)

Added dataset Root (ca41053a-62db-44fd-b1d6-0afbc9eaf81a)
Added dataset Root (7cea4a7a-ba68-47d7-9541-b48acffd2182)


In [9]:
# Lets see if it worked
print(db.connection.table("Root"))
print(db.connection.table("Root_nested"))
print(db.connection.table("Root_some_values"))

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ Root_id                              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│    20.0 │ 7cea4a7a-ba68-47d7-9541-b48acffd2182 │
│   100.0 │ ca41053a-62db-44fd-b1d6-0afbc9eaf81a │
└─────────┴──────────────────────────────────────┘



┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value     ┃ value   ┃ Root_nested_id                       ┃ Root_id                              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ float64 │ !string                              │ string                               │
├───────────────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ hi there          │     nan │ 2e67cd7a-4a17-460d-8004-1ef46e57ff38 │ 7cea4a7a-ba68-47d7-9541-b48acffd2182 │
│ this is dataset 2 │     nan │ 5c9452b1-f494-4b3d-8b4c-2be1f41cb026 │ 7cea4a7a-ba68-47d7-9541-b48acffd2182 │
│ something         │     nan │ 695bca66-4c00-4458-8b71-fb1fe9feed55 │ ca41053a-62db-44fd-b1d6-0afbc9eaf81a │
│ something else    │     nan │ 87ea12f0-92f7-4e0d-a8b8-856c317b33d6 │ ca41053a-62db-44fd-b1d6-0afbc9eaf81a │
└─────────

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ some_values ┃ Root_id                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string     │ string                               │
├─────────────┼──────────────────────────────────────┤
│ Hello       │ ca41053a-62db-44fd-b1d6-0afbc9eaf81a │
│ World       │ ca41053a-62db-44fd-b1d6-0afbc9eaf81a │
│ Whats       │ 7cea4a7a-ba68-47d7-9541-b48acffd2182 │
│ up          │ 7cea4a7a-ba68-47d7-9541-b48acffd2182 │
└─────────────┴──────────────────────────────────────┘



In [10]:
# Finally, recover the data from the database by filtering
retrieved = db.get(table="Root", attribute="value", value=100.0,)
print(retrieved[0])

Root
├── id = ca41053a-62db-44fd-b1d6-0afbc9eaf81a
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = 695bca66-4c00-4458-8b71-fb1fe9feed55
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = 87ea12f0-92f7-4e0d-a8b8-856c317b33d6
│           └── another_value = something else
└── some_values = [Hello, World, ...]

